In [86]:
# Apply the code on spam data set available in the source code (text classification on the spam.csv data set)

In [87]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

from sklearn.preprocessing import LabelEncoder

In [88]:
# load data
data = pd.read_csv('data/spam.csv', encoding='latin-1')
# Keeping only the neccessary columns
data = data[['v2','v1']]

data['v2'] = data['v2'].apply(lambda x: x.lower())
data['v2'] = data['v2'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))
data.head()

,v2,v1
0,go until jurong point crazy available only in ...,ham
1,ok lar joking wif u oni,ham
2,free entry in 2 a wkly comp to win fa cup fina...,spam
3,u dun say so early hor u c already then say,ham
4,nah i dont think he goes to usf he lives aroun...,ham


In [89]:
for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

In [90]:
# tokenization
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['v2'].values)
X = tokenizer.texts_to_sequences(data['v2'].values)

X = pad_sequences(X)

In [91]:
embed_dim = 128
lstm_out = 196

In [92]:
def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim, input_length=X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # print(model.summary())
    return model

In [94]:
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['v1'])
y = to_categorical(integer_encoded)

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.33, random_state=42)
batch_size = 32

model = createmodel()
model.fit(X_train, Y_train,  epochs = 5, batch_size=batch_size, verbose=2)
score, acc = model.evaluate(X_test, Y_test, verbose=2, batch_size=batch_size)
print(model.metrics_names[0], ":", score)
print(model.metrics_names[1], ":", acc)

Epoch 1/5
117/117 - 103s - loss: 0.1795 - accuracy: 0.9387
Epoch 2/5
117/117 - 69s - loss: 0.0403 - accuracy: 0.9877
Epoch 3/5
117/117 - 61s - loss: 0.0227 - accuracy: 0.9930
Epoch 4/5
117/117 - 60s - loss: 0.0099 - accuracy: 0.9976
Epoch 5/5
117/117 - 61s - loss: 0.0073 - accuracy: 0.9979
58/58 - 4s - loss: 0.0875 - accuracy: 0.9821
loss : 0.08748931437730789
accuracy : 0.9820554852485657
